# Current Issue

When swapping the agent to use the vision, it dislike the observation space where it uses the camera, since the rendering output of the camera is `('walker/egocentric_camera', BoundedArray(shape=(64, 64, 3), dtype=dtype('uint8')`, but it expect a float.

> Fixed! This is achieved by the new wrapper that converts the uint8 to float32, to be consistent with the rest of the observation spaces.

> New Error

```python
2024-07-19 07:59:43,188 ERROR worker.py:409 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::EnvironmentLoop.run() (pid=2623445, ip=10.244.10.88, actor_id=7619541ce6bdcfe57077c81e01000000, repr=<flybody.agents.ray_distributed_dmpo.EnvironmentLoop object at 0x7f1883744be0>)
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/acme/environment_loop.py", line 158, in run
    result = self.run_episode()
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/acme/environment_loop.py", line 94, in run_episode
    action = self._actor.select_action(timestep.observation)
  File "/root/vast/scott-yang/flybody/flybody/agents/actors.py", line 79, in select_action
    action = self._policy(observation)
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/tensorflow/python/framework/func_graph.py", line 1147, in autograph_handler
    raise e.ag_error_metadata.to_exception(e)
TypeError: in user code:

    File "/root/vast/scott-yang/flybody/flybody/agents/actors.py", line 65, in _policy  *
        policy = self._policy_network(batched_observation)
    File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/sonnet/src/utils.py", line 85, in _decorate_unbound_method  *
        return decorator_fn(bound_method, self, args, kwargs)
    File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/sonnet/src/base.py", line 262, in wrap_with_name_scope  *
        return method(*args, **kwargs)
    File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/sonnet/src/sequential.py", line 68, in __call__  *
        outputs = mod(outputs, *args, **kwargs)
    File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/sonnet/src/utils.py", line 85, in _decorate_unbound_method  *
        return decorator_fn(bound_method, self, args, kwargs)
    File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/sonnet/src/base.py", line 262, in wrap_with_name_scope  *
        return method(*args, **kwargs)
    File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/acme/tf/utils.py", line 144, in __call__  *
        return self._transformation(*args, **kwargs)
    File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/acme/tf/utils.py", line 54, in batch_concat  *
        return tf.concat(tree.flatten(flat_leaves), axis=-1)

    TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [float32, float32, float32, uint8, float32, float32, float32, float32, float32, float32, float32, float32, float32, float32, float32, float32] that don't all match.
```

# Issue (as of Jul 19)

- two touch and maze forage is not training. 
- Get those enviornment training!
- seems like some enviornment have integer task logic gate, and the neural network is not happy.

- TODO: wandb logging with ray,
- TODO: ask Salk support for Ray distributed training.
- maybe 0.4 gpu for training



> - TODO: environment factory is a function that is being passed into the env loop. So we need functool.partial for this.



# Jul 25 Issue: 

When trying to train a generalist, we found that
```python
(Learner pid=623126) None
Error executing job with overrides: []
Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 919, in ray._raylet.prepare_args_internal
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/_private/serialization.py", line 519, in serialize
    return self._serialize_to_msgpack(value)
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/_private/serialization.py", line 497, in _serialize_to_msgpack
    pickle5_serialized_object = self._serialize_to_pickle5(
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/_private/serialization.py", line 444, in _serialize_to_pickle5
    raise e
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/_private/serialization.py", line 439, in _serialize_to_pickle5
    inband = pickle.dumps(
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/cloudpickle/cloudpickle.py", line 1479, in dumps
    cp.dump(obj)
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/cloudpickle/cloudpickle.py", line 1245, in dump
    return super().dump(obj)
TypeError: cannot pickle 'weakref.ReferenceType' object

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/root/vast/scott-yang/flybody/flybody/train_dmpo_ray.py", line 262, in main
    actors.append(create_actors(num_actors, environment_factories[name]))
  File "/root/vast/scott-yang/flybody/flybody/train_dmpo_ray.py", line 243, in create_actors
    actor = EnvironmentLoop.remote(
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/actor.py", line 718, in remote
    return self._remote(args=args, kwargs=kwargs, **self._default_options)
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/util/tracing/tracing_helper.py", line 388, in _invocation_actor_class_remote_span
    return method(self, args, kwargs, *_args, **_kwargs)
  File "/root/anaconda3/envs/flybody/lib/python3.10/site-packages/ray/actor.py", line 1161, in _remote
    actor_id = worker.core_worker.create_actor(
  File "python/ray/_raylet.pyx", line 4111, in ray._raylet.CoreWorker.create_actor
  File "python/ray/_raylet.pyx", line 4116, in ray._raylet.CoreWorker.create_actor
  File "python/ray/_raylet.pyx", line 878, in ray._raylet.prepare_args_and_increment_put_refs
  File "python/ray/_raylet.pyx", line 869, in ray._raylet.prepare_args_and_increment_put_refs
  File "python/ray/_raylet.pyx", line 928, in ray._raylet.prepare_args_internal
TypeError: Could not serialize the argument <acme.wrappers.canonical_spec.CanonicalSpecWrapper object at 0x7f4aac1ea7a0> for a task or actor flybody.agents.ray_distributed_dmpo.EnvironmentLoop.__init__:
================================================================================
Checking Serializability of <acme.wrappers.canonical_spec.CanonicalSpecWrapper object at 0x7f4aac1ea7a0>
================================================================================
!!! FAIL serialization: cannot pickle 'weakref.ReferenceType' object
    Serializing '_abc_impl' <_abc._abc_data object at 0x7f50ec2f4480>...
    !!! FAIL serialization: cannot pickle '_abc._abc_data' object
    WARNING: Did not find non-serializable object in <_abc._abc_data object at 0x7f50ec2f4480>. This may be an oversight.
================================================================================
Variable: 

        FailTuple(_abc_impl [obj=<_abc._abc_data object at 0x7f50ec2f4480>, parent=<acme.wrappers.canonical_spec.CanonicalSpecWrapper object at 0x7f4aac1ea7a0>])

was found to be non-serializable. There may be multiple other undetected variables that were non-serializable. 
Consider either removing the instantiation/imports of these variables or moving the instantiation into the scope of the function/class. 
================================================================================
Check https://docs.ray.io/en/master/ray-core/objects/serialization.html#troubleshooting for more information.
If you have any suggestions on how to improve this error message, please reach out to the Ray developers on github.com/ray-project/ray/issues/
================================================================================
```

Might due to my environment factory function is not serializable. Might need to write multiple function to adapt to this scheme.

# July 26 

Task switching works, need to align observation spaces between task, it won't be a issue for the decoder encoder arch, where we shared the decoder but not the encoder.

**Observation Space Differences:**

- Run Gaps 
    - (base)
- Bowl Escape 
    - (base + origin coordinates)
- Maze Forage 
    - (base)
- Two Taps 
    - (base + task logic)

To enable joint training, we need to modify the observation space of all task to be the same.

_Question: At which layer does can we efficiently and reusably inject this observation change?_

> **(1):** Change in network (easy to implement but hard to load the checkpoint) 
>
> **(2):** Change the definition of tasks (more invovled implementation but it is a one time for all change)